In [1]:
import cv2 as cv
import numpy as np
import os
import pyautogui
from scipy import ndimage
import tensorflow as tf

In [2]:
path = './'
video_name = 'test_video.mp4'
tenhon_name = 'tenhon.png'
tenhon_name2 = 'tenhon2.png'
global rc
rc = 200
global red
red = (0,0,255)
global green
green = (0,255,0)
global blue
blue = (255,0,0)
global marking
marking = False
showing = False
zsize = 90

In [3]:
def color_range(str):
    
    if str == "blue":
        l = np.array([rc,0,0])
        
    elif str == "green":
        l = np.array([0, rc, 0])
    elif str == "red":
        l = np.array([0, 0, rc])

    u = np.array([255,255,255])

    return l, u

In [4]:
def show(img):
    cv.imshow("title", img)
        
    if cv.waitKey(1) == ord('q'):
        cv.destroyAllWindows()
        return True
    else:
        return False
        

In [5]:
def show_img(img):
    cv.imshow("title", img)
    cv.waitKey()
    cv.destroyAllWindows()

In [6]:
def color_fliter(img, string):
    l, u = color_range(string)
    mask = cv.inRange(img, l, u)
    img = cv.bitwise_or(img, img, mask=mask)
    return img


In [7]:
def find_contours(img):

    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

    im_thresh = cv.adaptiveThreshold(gray, 255, cv.ADAPTIVE_THRESH_MEAN_C, cv.THRESH_BINARY, 21, 0)

    return cv.findContours(im_thresh, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

In [8]:
def draw_rectangle(img, x, y, w, h):
    cv.rectangle(img, (x,y), (x+w, y+h), green, 3)

In [9]:
def draw_text(img, x, y, string):
    cv.putText(img, string, (x, y), cv.FONT_HERSHEY_PLAIN, 1, red, 1, cv.LINE_AA)

In [17]:
"""
while True:

    screenshot = pyautogui.screenshot()
    screenshot = np.array(screenshot)
    screenshot = cv.cvtColor(screenshot, cv.COLOR_RGB2BGR)
    cv.imshow('', screenshot)

    if cv.waitKey(1) == ord('q'):
        cv.destroyAllWindows()
        break

print('Done')
"""

"\nwhile True:\n\n    screenshot = pyautogui.screenshot()\n    screenshot = np.array(screenshot)\n    screenshot = cv.cvtColor(screenshot, cv.COLOR_RGB2BGR)\n    cv.imshow('', screenshot)\n\n    if cv.waitKey(1) == ord('q'):\n        cv.destroyAllWindows()\n        break\n\nprint('Done')\n"

In [10]:
from tensorflow.keras.callbacks import Callback

In [22]:
class CustomCallback(Callback):

    def __init__(self, patience=0):
        super(CustomCallback, self).__init__()
        self.patience = patience
        self.best_weights = None

    def on_train_begin(self, logs=None): 
        self.wait = 0
        self.stopped_epoch = 0
        self.best = np.Inf

    def on_epoch_end(self, epoch, logs=None): 
        current_loss = logs.get("loss")
        current_accuracy = logs.get("accuracy")
        if np.less(current_loss, self.best) :
            self.best = current_loss
            self.wait = 0
            self.best_weights = self.model.get_weights() 

            if current_loss < 1e-4 and current_accuracy == 1.0:
                self.stopped_epoch = epoch
                self.model.stop_training = True
                self.model.set_weights(self.best_weights)

        else:
            self.wait += 1
            if self.wait >= self.patience:
                self.stopped_epoch = epoch
                self.model.stop_training = True
                self.model.set_weights(self.best_weights)

    def on_train_end(self, logs=None):
        if self.stopped_epoch > 0:
            print("Epoch %05d: early stopping" % (self.stopped_epoch + 1))

In [18]:
img_name = os.listdir(path+"/orignal_data")

img_name = [i for i in img_name if "png" in i and "tenhon" in i]

sol_dict = dict()

f = open(path+"/orignal_data/tenhon_sol.txt", 'r')

for name, line in zip(img_name, f.readlines()):
    line = line.replace("\n","").split(" ")
    sol_dict[name] = line

f.close()

In [22]:
train_path = path + "/train_data/"

for file_name in sol_dict.keys():

    img = cv.imread("./orignal_data/" + file_name)
    
    mask_img = color_fliter(img, "blue")
    
    contour, hierarchy = find_contours(mask_img)

    contour_list = list()

    for pos in range(len(contour)):
        
        x, y, w, h = cv.boundingRect(contour[pos])

        if ((w > 20 and w < 100 and h > 40 and h < 150) or (h > 20 and h < 100 and w > 40 and w < 150)):

            contour_list.append([x,y,w,h,contour[pos]])

            if marking:
                draw_rectangle(img, x, y, w, h)
            
        if showing:
            show(img)

    contour_list.sort(key=lambda x : (x[1], x[0]))

    for (x,y,w,h,_), string in zip(contour_list, sol_dict[file_name]):
        
        if string in os.listdir(train_path):
            continue
        else:
            pe = train_path + "/" + string
            os.mkdir(pe)

            part = img[y:y+h, x:x+w]

            cv.imwrite(pe+"/"+string + "ori" + ".png", part)

            count = 1

            for _ in range(0,4):
                
                for xn in range(-4, 4, 2):
                    for yn in range(-6, 6, 2):

                        if len(string) > 2:

                            if string[2] == "r" and (abs(xn) > 4 or abs(yn) > 6):
                                continue

                        if x > y:
                            temp = yn
                            yn = xn
                            xn = temp

                        w = part.shape[1]
                        h = part.shape[0]
                        
                        p = part[np.where(yn > 0, yn, 0):np.where(yn > 0, h, h+yn), np.where(xn > 0, xn, 0):np.where(xn > 0, w, w+xn)]

                        for size in range(0,50,10):
                            ps = cv.resize(p, (int(p.shape[1] * ((100-size)/100)), int(p.shape[0] * ((100-size)/100))))

                            cv.imwrite(pe+"/"+string + str(count) + ".png", ps)

                            count+= 1

                part = cv.rotate(part, cv.ROTATE_90_CLOCKWISE)


In [18]:
train_path = path + "/train_data/"

if "ztr" not in os.listdir(train_path):
    os.mkdir(train_path + "/ztr")

img = cv.imread(path + "orignal_data/dumn1.png")

mask_img = color_fliter(img, "blue")

contour, hierarchy = find_contours(mask_img)

contour_list = list()

for pos in range(len(contour)):
        
        x, y, w, h = cv.boundingRect(contour[pos])

        if ((w > 20 and w < 100 and h > 40 and h < 150) or (h > 20 and h < 100 and w > 40 and w < 150)):

            contour_list.append([x,y,w,h,contour[pos]])


for count, (x,y,w,h,_) in enumerate(contour_list):
    
    pe = train_path + "/" + "ztr"
    part = img[y:y+h, x:x+w]
    cv.imwrite(pe+"/"+str(count)+".png", part)


In [76]:
train_path = path + "/train_data/"

img = cv.imread(path + "orignal_data/dumn2.png")

mask_img = color_fliter(img, "blue")

contour, hierarchy = find_contours(mask_img)

contour_list = list()

count = 0

xy_list = list()

for pos in range(len(contour)):
    
    x, y, w, h = cv.boundingRect(contour[pos])

    if ((w > 20 and w < 100 and h > 40 and h < 150) or (h > 20 and h < 100 and w > 40 and w < 150)):

        # contour_list.append(contour[pos])

        part = img[y+int(h*0.1):y+int(h*0.9), x+int(w*0.1):x+int(w*0.9)]

        part = cv.resize(part, (zsize, zsize))

        contour_list.append(part)
        xy_list.append([x,y,w,h])

        # if marking:

        #     draw_rectangle(frame, x,y,w,h)

if len(contour_list) > 0:

    k = im_model.predict(np.array(contour_list), verbose=0)

    k = np.argmax(k, axis=1)

    for (x,y,w,h),label in zip(xy_list,k):

        cv.rectangle(img, (x,y), (x+w, y+h), green, 3)
        cv.putText(img, label_dict[label], (x, y), cv.FONT_HERSHEY_PLAIN, 2, red, 1, cv.LINE_AA)

cv.imshow("title", img)
cv.waitKey()
cv.destroyAllWindows()

In [17]:

str_dict = """
1s = 0
2s = 1
3s = 2
4s = 3
5s = 4
6s = 5
7s = 6
8s = 7
9s = 8
5sr = 9
1m = 10
2m = 11
3m = 12
4m = 13
5m = 14
6m = 15
7m = 16
8m = 17
9m = 18
5mr = 19
1p = 20
2p = 21
3p = 22
4p = 23
5p = 24
6p = 25
7p = 26
8p = 27
9p = 28
5pr = 29
ew = 30
sw = 31
ww = 32
nw = 33
wd = 34
gd = 35
rd = 36
ztr = 37
"""

str_dict = "{" + str_dict.replace(" ","").replace("=","\":").replace("\n", ",\"")[1:-2] + "}"

str_dict = eval(str_dict)

if "train_data" not in os.listdir(path):
    os.mkdir(path+"/train_data")

In [16]:
train_path = path+"/"+"train_data"

file_names = os.listdir(train_path)

In [12]:
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, BatchNormalization, Input, Flatten, Conv1D, MaxPool1D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [13]:
def nn_model(zsize):
    input_layer = Input((zsize,zsize,3))
    conv_layer = Conv2D(16, (2,2), activation='relu')(input_layer)
    max_layer = MaxPool2D((2,2))(conv_layer)
    # bn_layer = BatchNormalization()(max_layer)
    conv_layer = Conv2D(32, (2,2), activation='relu')(max_layer)
    max_layer = MaxPool2D((2,2))(conv_layer)
    # bn_layer = BatchNormalization()(max_layer)
    conv_layer = Conv2D(64, (2,2),  activation='relu')(max_layer)
    max_layer = MaxPool2D((2,2))(conv_layer)
    bn_layer = BatchNormalization()(max_layer)
    conv_layer = Conv2D(128, (2,2),  activation='relu')(bn_layer)
    max_layer = MaxPool2D((2,2))(conv_layer)
    # bn_layer = BatchNormalization()(max_layer)
    # conv_layer = Conv2D(128, (2,2),  activation='relu')(bn_layer)
    # max_layer = MaxPool2D((2,2))(conv_layer)
    flatten_layer = Flatten()(max_layer)
    fc_layer = Dense(76, activation='relu')(flatten_layer)
    fc_layer = Dense(38, activation='softmax')(fc_layer)

    model = Model(input_layer, fc_layer)
    model.compile(optimizer=Adam(1e-3), loss='sparse_categorical_crossentropy', metrics="accuracy")
    
    return model



In [58]:
def gray_nn_model(zsize):
    input_layer = Input((zsize,zsize))
    conv_layer = Conv1D(16, 2, activation='relu')(input_layer)
    max_layer = MaxPool1D(2)(conv_layer)
    bn_layer = BatchNormalization()(max_layer)
    conv_layer = Conv1D(32, 2, activation='relu')(bn_layer)
    max_layer = MaxPool1D(2)(conv_layer)
    bn_layer = BatchNormalization()(max_layer)
    conv_layer = Conv1D(64, 2, activation='relu')(bn_layer)
    max_layer = MaxPool1D(2)(conv_layer)
    bn_layer = BatchNormalization()(max_layer)
    conv_layer = Conv1D(64, 2, activation='relu')(bn_layer)
    max_layer = MaxPool1D(2)(conv_layer)
    # bn_layer = BatchNormalization()(max_layer)
    flatten_layer = Flatten()(max_layer)
    fc_layer = Dense(38, activation='relu')(flatten_layer)
    # bn_layer = BatchNormalization()(fc_layer)
    fc_layer = Dense(38, activation='softmax')(fc_layer)
    
    model = Model(input_layer, fc_layer)
    model.compile(optimizer=Adam(1e-3), loss='sparse_categorical_crossentropy', metrics="accuracy")
    
    return model


In [27]:
im_model = nn_model(zsize)
im_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 90, 90, 3)]       0         
                                                                 
 conv2d (Conv2D)             (None, 89, 89, 16)        208       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 44, 44, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 43, 43, 32)        2080      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 21, 21, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 20, 20, 64)        8256  

In [18]:
image_list = list()
label_list = list()


for name in file_names:

    if ".txt" in name:
        continue

    sub_list = os.listdir(train_path + "/" + name)
    
    for sub in sub_list:
        p = cv.imread(train_path + "/" + name + "/" + sub)
        p = cv.resize(p, (zsize,zsize))

        image_list.append(p)
        label_list.append(str_dict[name])

        
    

In [15]:
noise_data = list()
noise_label = list()

for _ in range(77500):

    n = np.random.randint(0,255,(zsize,zsize,3))
    # n = np.where(n > 125, 255, 0)

    noise_data.append(n)
    noise_label.append(37)

In [23]:
im_model = nn_model(zsize)

# im_model.compile(optimizer=Adam(1e-5), loss='sparse_categorical_crossentropy', metrics="accuracy")

with tf.device('/GPU:1'):

    # im_model.fit(np.array(image_list + noise_data), np.array(label_list + noise_label), batch_size=1024, epochs=300, callbacks=[CustomCallback()])
    # im_model = nn_model(zsize)
    im_model.fit(np.array(image_list), np.array(label_list), batch_size=256, epochs=300, callbacks=[CustomCallback()])

Epoch 1/300
68/68 [==============================] - 29s 419ms/step - loss: 0.9038 - accuracy: 0.7862
Epoch 2/300
68/68 [==============================] - 29s 420ms/step - loss: 0.0166 - accuracy: 0.9998
Epoch 3/300
68/68 [==============================] - 28s 415ms/step - loss: 0.0043 - accuracy: 1.0000
Epoch 4/300
68/68 [==============================] - 31s 459ms/step - loss: 0.0023 - accuracy: 1.0000
Epoch 5/300
68/68 [==============================] - 33s 478ms/step - loss: 0.0014 - accuracy: 1.0000
Epoch 6/300
68/68 [==============================] - 32s 478ms/step - loss: 0.0010 - accuracy: 1.0000
Epoch 7/300
68/68 [==============================] - 33s 485ms/step - loss: 7.4765e-04 - accuracy: 1.0000
Epoch 8/300
68/68 [==============================] - 32s 476ms/step - loss: 5.7275e-04 - accuracy: 1.0000
Epoch 9/300
68/68 [==============================] - 33s 489ms/step - loss: 4.5328e-04 - accuracy: 1.0000
Epoch 10/300
68/68 [==============================] - 33s 483ms/step -

In [24]:
label_dict = dict((v,k) for k, v in str_dict.items())

cap = cv.VideoCapture(path+video_name)

while (cap.isOpened()):

    ret, frame = cap.read()

    mask_img = color_fliter(frame, "blue")
    
    contour, hierarchy = find_contours(mask_img)

    contour_list = list()

    xy_list = list()

    # img_gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

    # img_gray = cv.Canny(img_gray, 220, 220)

    # img_gray = np.where(img_gray > 200 , 255, img_gray)
    
    for pos in range(len(contour)):
        
        x, y, w, h = cv.boundingRect(contour[pos])

        if ((w > 20 and w < 100 and h > 40 and h < 150) or (h > 20 and h < 100 and w > 40 and w < 150)):

            # contour_list.append(contour[pos])

            # part = img_gray[y+int(h*0.1):y+int(h*0.9), x+int(w*0.1):x+int(w*0.9)]
            part = frame[y:y+h, x:x+w]

            # part = cv.Canny(part, 255,255)

            part = cv.resize(part, (zsize, zsize))

            contour_list.append(part)
            xy_list.append([x,y,w,h])

            # if marking:

            #     draw_rectangle(frame, x,y,w,h)

    if len(contour_list) > 0:

        k = im_model.predict(np.array(contour_list), verbose=0)

        k = np.argmax(k, axis=1)

        count = 0

        for (x,y,w,h),label in zip(xy_list,k):

            if label < 37:

                cv.rectangle(frame, (x,y), (x+w, y+h), green, 3)
                cv.putText(frame, label_dict[label], (x, y), cv.FONT_HERSHEY_PLAIN, 2, red, 1, cv.LINE_AA)

    isBreak = show(frame)
        
    if isBreak:
        break
    

In [106]:
import numpy as np
import cv2
import math


#read image
img = cv2.imread("./train_data/1m/1m.png")

# change size of image
width = int(img.shape[1])
height = int(img.shape[0])
dim = (width, height)

min_color = int(np.max(img))
min_color = (min_color, min_color, min_color)

# img = cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE)

#2d to 3d (projection)  , and -> rotation point - center point (origin point)
proj2dto3d = np.array([[1,0,-img.shape[1]],
                      [0,1,-img.shape[0]],
                      [0,0,0],
                      [0,0,1]],np.float32)

# 3d matrixs in  x ,y ,z 

'''
 you can remove any matrix if you dont want to rotate around it , so in our case 
 we rotate around y axis only so any line ends with " #0 " we can remove it 
 and the programe will run 

'''
rx   = np.array([[1,0,0,0],
                 [0,1,0,0],
                 [0,0,1,0],
                 [0,0,0,1]],np.float32)

ry   = np.array([[1,0,0,0],
                 [0,1,0,0],
                 [0,0,1,0],
                 [0,0,0,1]],np.float32)

rz   = np.array([[1,0,0,0],
                 [0,1,0,0],
                 [0,0,1,0],
                 [0,0,0,1]],np.float32)


trans= np.array([[1,0,0,0],
                 [0,1,0,0],
                 [0,0,1,height+20],
                 [0,0,0,1]],np.float32)



proj3dto2d = np.array([ [width,0,img.shape[1],0],
                        [0,height,img.shape[0],0],
                        [0,0,1,0] ],np.float32)

x = -40.0 #0
y = 0.0
z = 0.0 #0


ax = float(x * (math.pi / 180.0)) #0
ay = float(y * (math.pi / 180.0)) 
az = float(z * (math.pi / 180.0)) #0

rx[1,1] = math.cos(ax) #0
rx[1,2] = -math.sin(ax) #0
rx[2,1] = math.sin(ax) #0
rx[2,2] = math.cos(ax) #0

ry[0,0] = math.cos(ay)
ry[0,2] = -math.sin(ay)
ry[2,0] = math.sin(ay)
ry[2,2] = math.cos(ay)

rz[0,0] = math.cos(az) #0
rz[0,1] = -math.sin(az) #0
rz[1,0] = math.sin(az) #0
rz[1,1] = math.cos(az) #0

r =rx.dot(ry).dot(rz) # if we remove the lines we put    r=ry
final = proj3dto2d.dot(trans.dot(r.dot(proj2dto3d)))

dst = cv2.warpPerspective(img, final,(img.shape[1],img.shape[0]),None,cv2.INTER_LINEAR
                            ,cv2.BORDER_CONSTANT,min_color)


cv2.imshow("dst",dst)

cv2.waitKey(0)

cv2.destroyAllWindows()

In [188]:
img = cv.imread("./dumn2.PNG")

mask_img = color_fliter(img, "blue")
    
contour, hierarchy = find_contours(mask_img)

contour_list = list()

xy_list = list()

# img_gray = cv.cvtColor(mask_img, cv.COLOR_BGR2GRAY)

# img_gray = cv.Canny(img_gray, 255, 255)

# img_gray = np.where(img_gray > 200 , 255, img_gray)

for pos in range(len(contour)):
    
    x, y, w, h = cv.boundingRect(contour[pos])

    if (w > 20 and w < 100 and h > 40 and h < 150) or (h > 20 and h < 100 and w > 40 and w < 150):

        # contour_list.append(contour[pos])

        # part = img_gray[y+int(h*0.1):y+int(h*0.9), x+int(w*0.1):x+int(w*0.9)]
        part = img[y:y+h, x:x+w]

        # part = cv.Canny(part, 255, 255)

        part = cv.resize(part, (zsize, zsize))

        contour_list.append(part)
        xy_list.append([x,y,w,h])

        # show_img(part)

k = im_model.predict(np.array(contour_list), verbose=0)

k = np.argmax(k, axis=1)


# img_gray = cv.cvtColor(img_gray, cv.COLOR_GRAY2BGR)

for ((x,y,w,h), label) in zip(xy_list,k):

    cv.rectangle(img, (x,y), (x+w, y+h), green, 3)
    cv.putText(img, label_dict[label], (x, y), cv.FONT_HERSHEY_PLAIN, 2, red, 1, cv.LINE_AA)

show_img(img)
            


In [ ]:
#git test